# Snek Demo

Snek is developed to improve upon the original cobra package by making it easier to access and less prone for unintended errors.

In [1]:
import cobra 
import snek
model = cobra.io.load_model('textbook')

def printer(x):
    print('{:5.2f}'.format(x))

## 1. Ensure Correct Spelling

When setting bounds of a reaction or the objective function with the main CobraPy package, no error is raised when a spelling error occurs.

In [2]:
with model:
    model.reactions.EX_glc__D_e.lower_bound = -5
    printer(model.slim_optimize())
    model.reactions.EX_glc__D_e.lower_buond = -10
    # no error is raised but bound is not updated.
    printer(model.slim_optimize())

 0.42
 0.42


This can happen frequently and is hard to debug. Also, the officially recommended way of setting exchange bounds needs 3 lines instead of 1.

In [3]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 5
    model.medium = medium
    printer(model.slim_optimize())
    medium["EX_glc__D_e"] = 10
    model.medium = medium
    printer(model.slim_optimize())

 0.42
 0.87


The following snek.core functions ensure correct spelling and are still one-liners for efficient coding.

In [4]:
with model:
    model = snek.set_bounds(model,'EX_glc__D_e',-5)
    printer(model.slim_optimize())
    model = snek.set_bounds(model,'EX_glc__D_e',-10)
    printer(model.slim_optimize())

 0.42
 0.87


The same issue can arise when trying to change the objective function of a model.

In [5]:
with model:
    printer(model.slim_optimize())
    model.objctive = 'EX_etoh_e'
    # no error is raised but objective function is not updated.
    printer(model.slim_optimize())
    model = snek.set_objective(model,'EX_etoh_e','max')
    printer(model.slim_optimize())

 0.87
 0.87
20.00


## 2. Checking for Logical Errors

CobraPy gives the user a lot of freedom. However, here we implemented some basic checks that ensure that no logical errors are made and prints warning if this is true.
This logical errors are sometimes intended, therefore only warnings are printed but the objective function is still updated. 
However, we think that a more verbose output can help researchers when exploring the model.

In [6]:
with model:
    model.objective = 'ATPM'
    model.objective_direction = 'min'
    printer(model.slim_optimize())
    # here we try to minimize a reaction with a custom lower bound
    # although this might be intended, it often isn't.
    model = snek.set_objective(model,'ATPM','min')
    printer(model.slim_optimize())

 8.39
 8.39


When the solution status is infeasible it is still possible to programmatically extract fluxes. This can lead to confusions while coding.

In [12]:
with model:
    model = snek.set_bounds(model,'EX_glc__D_e',0,0)
    solution = model.optimize()
    print(solution.status)
    printer(solution.fluxes['EX_glc__D_e'])

infeasible
-0.48


/home/users/mgotsmy/.conda/envs/2206test3.10/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


The snek optimization function checks for infeasibility and raises an error, so no confusions can happen.

In [13]:
with model:
    model = snek.set_bounds(model,'EX_glc__D_e',0,0)
    solution = snek.sensitive_optimize(model)
    print(solution.status)
    printer(solution.fluxes['EX_glc__D_e'])

/home/users/mgotsmy/.conda/envs/2206test3.10/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


ValueError: Solution is infeasible

## 3. Easy Programmatic Access

Some functionalities can be set very easily by strings, however, when you want the value that was entered returned you get a programmatically hard to deal with object back.

In [33]:
with model:
    model.solver = 'cplex'
    print(str(model.solver)[:100],'\n\n...\n',str(model.solver)[-100:])

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 baaa60b8m0678m11edmbd80m63dbbcd05b9c#0: Biomass_Ecol 

...
 
 0 <= TKT2_reverse_7ebc7#187 <= 1000
 0 <= TPI#188 <= 1000
 0 <= TPI_reverse_c2c3b#189 <= 1000
End



compared to 

In [38]:
with model:
    model.solver = 'cplex'
    print(snek.get_solver(model))

cplex


In [40]:
with model:
    model.objective = 'Biomass_Ecoli_core'
    print(str(model.objective.expression))

1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


compared to 

In [41]:
with model:
    model.objective = 'Biomass_Ecoli_core'
    print(snek.get_objective(model))

Biomass_Ecoli_core


The summary command calculates the C fluxes, however they cannot be easily programmatically accessed.

In [45]:
with model:
    summary = model.summary()
summary

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [47]:
summary.uptake_flux
# the C-flux is gone

,flux,reaction,metabolite
EX_glc__D_e,10.000000,EX_glc__D_e,glc__D_e
EX_nh4_e,4.765319,EX_nh4_e,nh4_e
EX_o2_e,21.799493,EX_o2_e,o2_e
EX_pi_e,3.214895,EX_pi_e,pi_e


With the snek.elements module the element wise fluxes can easily be accessed. Moreover, all mass non-balanced fluxes + their absolut values are included.

In [48]:
snek.elements.element_fluxes(model,'C')

,C_flux,C_flux%
EX_glc__D_e,60.000000,100.000000
EX_co2_e,-22.809833,-38.016389
Biomass_Ecoli_core,-37.190167,-61.983611


Moreover, not only C fluxes can be accessed. 

In [50]:
snek.elements.element_fluxes(model,'H')

,H_flux,H_flux%
EX_glc__D_e,120.000000,84.343006
EX_nh4_e,19.061277,13.397378
EX_pi_e,3.214895,2.259616
EX_h_e,-17.530865,-12.321716
EX_h2o_e,-58.351654,-41.012949
Biomass_Ecoli_core,-66.393652,-46.665335
